In [ ]:
import numpy as np

class MTIterator2D:
    def __init__(self, a, blocksize, axis=0):
        self.origa = a
        self.rows = self.origa.shape[0]
        self.cols = self.origa.shape[1]
        self.blocksize = blocksize
        self.axis= axis

        self.a = self.to_blocks()    
     
    def __getitem__(self,i):
        return self.a[i]

    def __setitem__(self,i,val):
        self.a[i] = val

    def shape(self,i):
        return a.shape[i]

    
    def to_blocks(self):
        '''convert the original matrix to an array of blocks'''
        squares = []

        for i in range(0, self.rows, self.blocksize):
            for j in range(0, self.cols, self.blocksize):
                square = self.origa[i:i+self.blocksize, j:j+self.blocksize]
                squares.append(square)

        allblocks = np.concatenate(squares).reshape(self.rows // self.blocksize, 
                                                    self.cols // self.blocksize, 
                                                    self.blocksize, 
                                                    self.blocksize)
        return allblocks


class BlockMatMul:
    def __init__(self):
        pass
    def __call__(self,a,b,c):
        return a @ b + c

class BlockMatMul:  
    def __init__(self):
        self.bmm = BlockMatMul()
        super.__init__()
    
    def callgraph(self, mema, memb, memc, blocksize):
        a, b, c = [MTIterator2D(m, blocksize) for m in [mema, memb, memc]]
        
        # Block Matrix Multiply with write to C once per block
        for i in range(a.shape[0]):
            for j in range(b.shape[1]):
                for k in range(a.shape[1]):
                    c[i,j] = bmm(a[i, k], b[k, j], c[i, j])
        
            memc[i,j] = c[i,j]


mema = np.random.randint(0, 10, size=(6, 6))
memb = np.random.randint(0, 10, size=(6, 6))
memc = np.zeros((mema.shape[0], memb.shape[1]))
block_size = 2


riallto_bmm = BlockMatMul()
riallto_bmm(mema,memb,memc)



In [1]:
for k in range(3):
    print(k)

0
1
2


In [ ]:
# https://github.com/Xilinx/mlir-aie/blob/main/programming_examples/basic/matrix_multiplication/single_core/aie2.py

# Need to iterate over A, B, C using memtile iterators and block matrix multiply


In [37]:
import numpy as np


class BlockMatMul:
    def __init__(self):
        pass
    def __call__(self,a,b,c):
        return np.dot(a,b) + c

class MTIterator1D:
    def __init__(self, a, axis=0):
        self.origa = a
        self.a = self.to_blocks()    
        self.axis= axis

    def __len__(self):
        if self.axis==0:
            return len(self.a)
        else:
            return len(self.a[0])        
   
    def __getitem__(self,i):
        if self.axis==0:
            return self.a[i, :]
        else:
            return self.a[:, i]

    def __setitem__(self,i,val):
        if self.axis==0:
            self.a[i, :] = val
        else:
            self.a[:, i] = val
    
    def to_blocks(self):
        block1 = self.origa[:2, :2]
        block2 = self.origa[:2, 2:]
        block3 = self.origa[2:, :2]
        block4 = self.origa[2:, 2:]
        return np.array([[block1, block2],[block3, block4]])

In [38]:
import numpy as np

mema = np.array([[1, 2, 3, 4],
                      [5, 6, 7, 8],
                      [9, 10, 11, 12],
                      [13, 14, 15, 16]])

memb = np.array([[1, 2, 3, 4],
                      [5, 6, 7, 8],
                      [9, 10, 11, 12],
                      [13, 14, 15, 16]])

memc = np.zeros(16).reshape(4,4)


bmm = BlockMatMul()
bmm(mema, memb, memc)

array([[ 90., 100., 110., 120.],
       [202., 228., 254., 280.],
       [314., 356., 398., 440.],
       [426., 484., 542., 600.]])

In [39]:
np.array_equal(np.dot(mema,memb),bmm(mema, memb, memc))

True

In [40]:
mema @ memb

array([[ 90, 100, 110, 120],
       [202, 228, 254, 280],
       [314, 356, 398, 440],
       [426, 484, 542, 600]])

In [41]:
import numpy as np

mema = np.random.randint(0, 10, size=(6, 6))
memb = np.random.randint(0, 10, size=(6, 6))
memc = np.zeros((A.shape[0], B.shape[1]))
block_size = 2

a = MTIterator2D(mema, blocksize)
b = MTIterator2D(memb, blocksize)
c = MTIterator2D(memc, blocksize)

# Iterate over blocks
for i in range(a.shape[0]):
    for j in range(b.shape[1]):
        for k in range(a.shape[1]):
            bmm(a[i, k], b[k, j], c[i, j])

    memc[i,j] = c[i,j]

ValueError: could not broadcast input array from shape (2,2,2,2) into shape (2,2,2)

In [27]:
np.dot(mema,mema)

array([[ 90, 100, 110, 120],
       [202, 228, 254, 280],
       [314, 356, 398, 440],
       [426, 484, 542, 600]])

In [50]:
import numpy as np

def block_matrix_multiply(A, B, block_size):
    # Check if the dimensions are compatible for block matrix multiplication
    if A.shape[1] != B.shape[0]:
        raise ValueError("The number of columns in A must be equal to the number of rows in B.")

    # Initialize the resulting matrix C with zeros
    C = np.zeros((A.shape[0], B.shape[1]))

    # Iterate over blocks
    for i in range(0, A.shape[0], block_size):
        for j in range(0, B.shape[1], block_size):
            for k in range(0, A.shape[1], block_size):
                # Calculate block multiplication
                C[i:i+block_size, j:j+block_size] += np.dot(A[i:i+block_size, k:k+block_size],
                                                             B[k:k+block_size, j:j+block_size])

    return C

# Example usage:
# Create two matrices A and B
A = np.random.randint(0, 10, size=(6, 6))
B = np.random.randint(0, 10, size=(6, 6))
block_size = 2

# Perform block matrix multiplication
C = block_matrix_multiply(A, B, block_size)

# Display the results
print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)
print("\nResulting Matrix C:")
print(C)




Matrix A:
[[4 2 3 2 3 4]
 [9 9 0 0 5 8]
 [1 5 0 3 3 5]
 [8 7 2 2 5 7]
 [4 3 5 8 3 2]
 [6 3 9 2 4 3]]

Matrix B:
[[8 0 5 7 3 1]
 [5 4 2 1 5 8]
 [5 4 0 4 7 5]
 [7 5 5 1 3 6]
 [6 7 0 2 1 5]
 [4 1 8 3 6 2]]

Resulting Matrix C:
[[105.  55.  66.  62.  76.  70.]
 [179.  79. 127. 106. 125. 122.]
 [ 92.  61.  70.  36.  70.  84.]
 [181.  88. 120. 104. 126. 125.]
 [154.  95.  82.  71. 101. 120.]
 [158.  89.  70. 100. 124. 113.]]


In [48]:
import numpy as np

def block_matrix_multiply(A, B, block_size):
    # Check if the dimensions are compatible for block matrix multiplication
    if A.shape[1] != B.shape[0]:
        raise ValueError("The number of columns in A must be equal to the number of rows in B.")

    # Initialize the resulting matrix C with zeros
    C = np.zeros((A.shape[0], B.shape[1]))

    # Iterate over blocks
    for i in range(0, A.shape[0], block_size):
        for j in range(0, B.shape[1], block_size):
            for k in range(0, A.shape[1], block_size):
                # Calculate block multiplication
                C[i:i+block_size, j:j+block_size] += np.dot(A[i:i+block_size, k:k+block_size],
                                                             B[k:k+block_size, j:j+block_size])

    return C

# Example usage:
# Create two matrices A and B
A = np.random.randint(0, 10, size=(6, 6))
B = np.random.randint(0, 10, size=(6, 6))
block_size = 2

# Perform block matrix multiplication
C = block_matrix_multiply(A, B, block_size)

# Display the results
print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)
print("\nResulting Matrix C:")
print(C)

print(np.array_equal(C, A @ B))

Matrix A:
[[7 3 8 9 3 7]
 [2 8 4 4 5 8]
 [5 4 0 1 6 6]
 [3 6 5 8 2 4]
 [0 7 2 6 4 1]
 [4 7 1 7 6 0]]

Matrix B:
[[8 0 5 3 9 4]
 [1 0 6 8 6 5]
 [9 3 8 8 6 7]
 [8 3 3 6 0 4]
 [1 1 5 4 0 6]
 [6 5 2 5 2 4]]

Resulting Matrix C:
[[248.  89. 173. 210. 143. 181.]
 [145.  69. 143. 186. 106. 154.]
 [ 94.  39.  94. 107.  81. 104.]
 [165.  61. 133. 173. 101. 137.]
 [ 83.  33.  98. 129.  56. 101.]
 [110.  30. 121. 142.  84. 122.]]
True


In [49]:
import numpy as np

def block_matrix_multiply(A, B, block_size):
    # Check if the dimensions are compatible for block matrix multiplication
    if A.shape[1] != B.shape[0]:
        raise ValueError("The number of columns in A must be equal to the number of rows in B.")

    # Initialize the resulting matrix C with zeros
    C = np.zeros((A.shape[0], B.shape[1]))

    # Iterate over blocks
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                # Calculate block multiplication
                C[i, j] += np.dot(A[i, k], B[k, j])

    return C


def block_matrix_multiply(A, B, C):

    for i in range(len(A)):
        for j in range(len(B)):
            for k in range(len(A[0]):
                C[i, j] += np.dot(A[i, k], B[k, j])

    return C




# Example usage:
# Create two matrices A and B
A = np.random.randint(0, 10, size=(6, 6))
B = np.random.randint(0, 10, size=(6, 6))
block_size = 2

# Perform block matrix multiplication
C = block_matrix_multiply(A, B, block_size)

# Display the results
print(np.array_equal(C, A @ B))



True
